# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,tessalit,20.20,1.01,90.99,29,15,4.05,ML
1,jasmine estates,28.29,-82.69,87.01,78,89,0.89,US
2,rikitea,-23.12,-134.97,68.38,73,38,7.24,PF
3,punta arenas,-53.15,-70.92,35.60,69,75,3.60,CL
4,albany,42.60,-73.97,96.01,41,63,4.06,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
index = city_data[(city_data["Humidity"] > 50) | \
                  (city_data["Wind Speed"] > 5) | \
                  (city_data["Max Temp"] < 70) | \
                  (city_data["Max Temp"] > 80) | \
                  (city_data["Cloudiness"] > 20)].index.tolist()
hotel_df = city_data.drop(index)
hotel_df = hotel_df.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
29,uberaba,-19.75,-47.93,75.20,29,0,3.60,BR
110,komsomolskiy,40.43,71.72,77.00,33,8,4.10,UZ
124,sayan,-11.14,-77.19,73.81,37,20,3.17,PE
218,torbay,47.67,-52.73,73.40,43,20,2.10,CA
319,oyotun,-6.85,-79.30,78.08,48,2,3.32,PE
339,semme,46.20,1.50,77.00,36,0,2.10,FR
441,montlucon,46.33,2.60,73.00,45,1,1.02,FR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df["Hotel"] = ""

In [16]:
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
      
for index, row in hotel_df.iterrows():
    params["location"] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    try:
        hotel_df.loc[index, "Hotel"] = results[0]["name"]
    except (KeyError, IndexError):
        print(f"There is no hotel within 5 km of {row['City']}")

There is no hotel within 5 km of torbay


In [17]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel
29,uberaba,-19.75,-47.93,75.20,29,0,3.60,BR,Hotel Dan Inn Uberaba
110,komsomolskiy,40.43,71.72,77.00,33,8,4.10,UZ,Hotel Mehmon Saroy
124,sayan,-11.14,-77.19,73.81,37,20,3.17,PE,Hotel Elizabeth Hoss
218,torbay,47.67,-52.73,73.40,43,20,2.10,CA,
319,oyotun,-6.85,-79.30,78.08,48,2,3.32,PE,Municipalidad Distrital de Oyotún
339,semme,46.20,1.50,77.00,36,0,2.10,FR,Hôtel Jinjaud
441,montlucon,46.33,2.60,73.00,45,1,1.02,FR,ibis Montluçon


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
